# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [6]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [37]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

# 1) What test is appropriate for this problem? Does CLT apply?

The best test to use would be hypothesis testing, and we would be using 2 samples since we are considering the data entries under black and white to be separate samples that need to be tested against each other in the population. Since we don't know the true population sigma, we can use the t-test. However, since the sample size seems pretty large, we can estimate the sigma by using the standard error of the sample and it will be very similar if not the same as the z-statistic.

We can use CLT in this situation if certain outcomes are met. We are assuming that the sample is truly random since the black and white assignments were random. The sample is less than 10% of the entire population of black and white job searchers in the US and the sample size is large enough to assume close to normality as can be done by multiplying the rates down below by the sample size to confirm that they are greater than 10.

# 2) What are the null and alternate hypotheses?

H<sub>0</sub> is that there is no statistically significant difference between the call back rates of the two groups.

H<sub>A</sub> is that there is a statistically significant difference in the call back rate of the two races

In [7]:
w = data[data.race=='w']
b = data[data.race=='b']

# 3)Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [9]:
# Your solution to Q3 here
b_rate = sum(b.call)/len(b)
w_rate = sum(w.call)/len(w)
print(b_rate,w_rate) #average rates of callback, aka the two means

0.06447638603696099 0.09650924024640657


In [20]:
import math
SE = math.sqrt((np.std(b.call)**2)/len(b))+(np.std(w.call)**2/len(w))
print(SE) #the standard error of the sample 

0.005012917146122524


In [21]:
#a 95% confidence interval of the difference based on the original sample. 
CI_low,CI_up = stats.norm.interval(0.95,loc=abs(b_rate-w_rate),scale=SE)
print(CI_low,CI_up)

0.022207717145562127 0.041857991273329044


In [60]:
ME = (abs(b_rate-w_rate)-CI_low) #the Margin of Error both using the confidence interval plus the standard error multiplied by the expected t-score from table at the expected 95% confidence interval
print(ME)
print(SE*1.96)

0.009011991279945161
0.009825317606400147


In [24]:
#doing a two sample t-test
from scipy.stats import ttest_ind
t_val, p_val = ttest_ind(b.call, w.call)
print('t-statistic: ',t_val)
print('p value: ',p_val)


t-statistic:  -4.114705290861751
p value:  3.940802103128886e-05


In [45]:
def draw_bs_reps(data, func, size=1):
    """Draw bootstrap replicates."""

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = func(np.random.choice(data, size=len(data)))

    return bs_replicates
#function for making bootstrap replicates


In [61]:
#Now let's bootstrap using a sampling size close to the original sample
#and run a two sample t-test again

w_reps = draw_bs_reps(np.array(w.call),np.mean,size=4000)
b_reps = draw_bs_reps(np.array(b.call),np.mean,size=4000)
t_val2, p_val2 = ttest_ind(b_reps, w_reps)
print('t-statistic: ',t_val2)
print('p value: ',p_val2)


t-statistic:  -339.2458570520456
p value:  0.0


In [57]:
# Compute the 95% confidence interval:
CI_low,CI_up = stats.norm.interval(0.95,loc=np.mean(abs(b_reps-w_reps)),scale=SE)
print(CI_low,CI_up) #slightly smaller than the frequentist test but only slightly

0.023020862929500424 0.04267113705726734


In [59]:
MOE = np.mean(abs(b_reps-w_reps))-CI_low
print(MOE) #and the margin of error from the bootstrap test. Smaller than frequentist test 

0.009825137063883459


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

## Write a story describing the statistical significance in the context or the original problem. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

Both in using the frequentist and bootstrap approches led to extremely low p-values, as low as zero for the bootstrap test. Therefore, the null hypothesis can be rejected. In the context of this problem this means that there is a statistically significant difference between teh call back rates between teh two races. Due to the calculations of the confidence intervals we are 95% confident that the true difference in the callbacks is between 2.2 and 4.18%.

However, this does not mean that this is the most important factor because the only comparison that we made was the callback rate and whether the name was white or black sounding. We were not ale to isolate any other factors based on the information given to us, nor did we utilize any other variables from teh data. To determine significance, it would be prudent to statistical analyses similar to the one just performed with other factors. Specifically, these could be resume related such as years experience or any special skills they had. Based on the only testing we have done, however, it is inconclusive whether it is the most significant factor or not. 